In [1]:
'''بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'''

'بِسْمِ اللهِ الرَّحْمٰنِ الرَّحِيْمِ'

In [2]:
'''سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'''

'سُبْحَـٰنَكَ لَا عِلْمَ لَنَآ إِلَّا مَا عَلَّمْتَنَآ ۖ إِنَّكَ أَنتَ الْعَلِيمُ الْحَكِيمُ'

In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    * {
        font-family: Monaco;
    }
</style>
""")

In [4]:
import os,sys,warnings,re,math,gc,time
warnings.filterwarnings("ignore")
os.cpu_count()

12

In [5]:
os.environ["POLARS_MAX_THREADS"] = "40"
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from glob import glob
from pathlib import Path
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates["mod"] = go.layout.Template(layout=dict(font=dict(family="Monaco",size=15)))
pio.templates.default = "mod+plotly_dark"
from sklearn.metrics import auc,roc_auc_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import miceforest as mf
from miceforest import mean_match_shap,mean_match_default,mean_match_fast_cat
%xmode Minimal

Exception reporting mode: Minimal


In [6]:
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_thousands_separator(",")
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(40)

polars.config.Config

In [7]:
path_to_train = Path("Downloads/credit_risk/train/")
path_to_test = Path("Downloads/credit_risk/test/")
path_to_features = Path("Downloads/feature_definitions.csv")

In [8]:
feat_df = pl.read_csv(path_to_features)
feat_df.head()

Variable,Description
str,str
"""actualdpd_943P""","""Days Past Due (DPD) of previous contract (actual)."""
"""actualdpdtolerance_344P""","""DPD of client with tolerance."""
"""addres_district_368M""","""District of the person's address."""
"""addres_role_871L""","""Role of person's address."""
"""addres_zip_823M""","""Zip code of the address."""


In [9]:
all_train_files = glob("Downloads/credit_risk/train/*.parquet")
all_test_files = glob("Downloads/credit_risk/test/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [10]:
train_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
7,"""Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""


In [11]:
train_base = (
    pl.scan_parquet(all_train_files[7])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["target","date"]),cs.contains("target"))
)
display(train_base.fetch().head())
train_base.collect().shape

case_id_base,month,week,weekday,target
u32,i8,i8,i8,u8
0,1,1,4,0
1,1,1,4,0
2,1,1,5,0
3,1,1,4,0
4,1,1,5,1


(1526659, 5)

In [12]:
test_files_df.filter(pl.col("filename").str.contains("base"))

index,path,filename
i64,str,str
33,"""Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [13]:
test_base = (
    pl.scan_parquet(all_test_files[33])
    .select(
        pl.col("case_id").cast(pl.UInt32).alias("case_id_base"),
        cs.contains("date").str.to_date().alias("date"),
    )
    .with_columns(
        pl.col("date").dt.month().alias("month"),
        pl.col("date").dt.week().alias("week"),
        pl.col("date").dt.weekday().alias("weekday")
    )

    .select(~cs.contains(["date"]),cs.contains("target"))
)
display(test_base.fetch().head())
print(test_base.collect().shape)

case_id_base,month,week,weekday
u32,i8,i8,i8
"57,543",10,41,2
"57,549",10,41,2
"57,551",10,41,2
"57,552",10,41,3
"57,569",10,41,2


(10, 4)


In [14]:
train_files_df.filter(pl.col("filename").str.contains("appl"))

index,path,filename
i64,str,str
31,"""Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


## Case_id, num_group1 and num_group2 relationships

<font size=6>

Goal of this competition is to predict risk quality of clients (or to be precise applications of clients), so client and his/her application is entity (case_id) for which your model predict probability of default (credit score).

- depth=0 (what we call static attributes) are attributes that are aggregated on case_id level. Example can be age of client or gender, 1 records per 1 case_id.

- depth=1, those are attributes where we have several records per client/application. Example can be previous applications or loans in credit bureau register, each client can have from 0 to n records. Therefore, for one case_id there might be several records, and to index them we use num_group1

- depth=2: For some attributes with depth=1 we have more detailed information, for example for previous applications we have data about instalments like date of payments or days past due of each payments. It means for each previous application, you can have 0…n records about instalments/payments. And as an index is used num_group2

To sum up, 1 client can have several previous applications, and each of those previous applications can have several records about instalments, payments, days past due, etc.

- num_group1 : 0 -> 19
- num_group2 : 0 -> 11

## Functions

In [15]:
def select_imputable_cols(df:pl.LazyFrame,threshold:int=10) -> pl.LazyFrame:
    col_list = (
        df
        .collect()
        .pipe(lambda df: (df.null_count()/df.shape[0])*100)
        .transpose(include_header=True,column_names=["percentage of nulls"])
        .join(
            feat_df,
            left_on="column",
            right_on="Variable",
            how="left"
        )
        .sort(by="percentage of nulls")
        .select(["column","Description","percentage of nulls"])
        .filter(pl.col("percentage of nulls") < threshold)
        ["column"].to_list()
    )
    _ = df.select(col_list)
    gc.collect()
    return _

In [16]:
def rename_cols(df: pl.LazyFrame) -> pl.LazyFrame:
    return (
        df
        .pipe(
            lambda df: df.rename(
                {_:_ + " " + feat_df.filter(pl.col("Variable") == _).select("Description").item() for _ in df.columns if _ not in ["case_id","num_group1","num_group2"]}
            )
        )
    )

In [17]:
train_past_depth_one = (
    pl.scan_parquet(path_to_train/"train_applprev_1_*.parquet",rechunk=True)
    .drop(cs.contains("date"))
    .select(
        pl.col("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical),
    )
)

In [18]:
train_past_depth_one.fetch().head(10)

case_id,num_group1,actualdpd_943P,annuity_853A,byoccupationinc_3656910L,childnum_21L,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_transactions_402L,credamount_590A,currdebt_94A,downpmt_134A,mainoccupationinc_437A,maxdpdtolerance_577P,outstandingdebt_522A,pmtnum_8L,revolvingaccount_394A,tenor_203L,cancelreason_3545846M,credacc_status_367L,credtype_587L,district_544M,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,inittransactioncode_279L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,status_219L
u32,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
2,0,0.00,640.20,null,0.00,null,0.00,null,null,null,"10,000.00",null,0.00,"8,200.00",null,null,24.00,null,24.00,"""a55475b1""",null,"""CAL""","""P136_108_173""",null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""
2,1,0.00,"1,682.40",null,0.00,null,0.00,null,null,null,"16,000.00",null,0.00,"8,200.00",null,null,12.00,null,12.00,"""a55475b1""",null,"""CAL""","""P136_108_173""",null,null,"""P97_36_170""","""2010-02-15""","""SINGLE""","""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""
3,0,0.00,"6,140.00",null,null,null,0.00,null,null,null,"59,999.80",null,0.00,"11,000.00",null,null,12.00,null,12.00,"""P94_109_143""",null,"""CAL""","""P131_33_167""",null,null,"""P97_36_170""","""2018-05-15""","""MARRIED""","""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D"""
4,0,0.00,"2,556.60",null,null,null,0.00,null,null,null,"40,000.00",null,0.00,"16,000.00",null,null,24.00,null,24.00,"""P24_27_36""",null,"""CAL""","""P194_82_174""",null,null,"""a55475b1""",null,null,"""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T"""
5,0,0.00,null,null,null,null,null,null,null,null,null,null,null,"62,000.00",null,null,null,null,null,"""P85_114_140""",null,null,"""P54_133_26""",null,null,"""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T"""
6,0,0.00,"1,773.80",null,null,null,0.00,null,null,null,"15,980.00",null,0.00,"37,000.00",null,null,11.00,null,11.00,"""P94_109_143""",null,"""CAL""","""P82_154_182""",null,null,"""a55475b1""",null,null,"""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D"""
6,1,0.00,"4,189.60",null,0.00,null,0.00,null,null,null,"32,000.00",null,0.00,"35,000.00",null,null,11.00,null,11.00,"""P94_109_143""",null,"""CAL""","""P82_154_182""",null,null,"""P97_36_170""","""2013-09-15""","""SINGLE""","""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D"""
6,2,0.00,"1,110.40",1.00,0.00,null,0.00,null,null,null,"17,380.00",0.00,0.00,"14,000.00",null,0.00,24.00,null,24.00,"""a55475b1""",null,"""COL""","""a55475b1""",null,null,"""P97_36_170""","""2012-09-15""","""SINGLE""","""POS""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D"""
10,0,0.00,"10,916.60",null,null,null,0.00,null,null,null,"100,000.00",null,0.00,"57,000.00",null,null,12.00,null,12.00,"""P73_130_169""",null,"""CAL""","""P38_65_49""",null,null,"""P17_36_170""",null,"""MARRIED""","""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T"""


In [19]:
train_past_depth_one_transformed = (
    train_past_depth_one
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .fill_null(strategy="mean")
    .group_by("case_id")
    .agg(
        cs.integer().min(),
        cs.float().max(),
        cs.categorical().mode().get(0)
    )
    .sort(by="case_id")
)
train_past_depth_one_transformed.fetch().head()

case_id,num_group1,actualdpd_943P Days Past Due (DPD) of previous contract (actual).,mainoccupationinc_437A Client's main income amount in their previous application.,credacc_credlmt_575A Credit card credit limit provided for previous applications.,credamount_590A Loan amount or card limit of previous applications.,downpmt_134A Previous application downpayment amount.,annuity_853A Monthly annuity for previous applications.,pmtnum_8L Number of payments made for the previous application.,tenor_203L Number of instalments in the previous application.,cancelreason_3545846M Application cancellation reason.,district_544M District of the address used in the previous loan application.,education_1138M Applicant's education level from their previous application.,postype_4733339M Type of point of sale.,profession_152M Profession of the client during their previous loan application.,rejectreason_755M Reason for previous application rejection.,rejectreasonclient_4145042M Reason for rejection of the client's previous application.,status_219L Previous application status.,credtype_587L Credit type of previous application.,inittransactioncode_279L Type of the initial transaction made in the previous application of the client.
u32,u8,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
2,0,0.00,"8,200.00",0.00,"16,000.00",0.00,"1,682.40",24.00,24.00,"""a55475b1""","""P136_108_173""","""P97_36_170""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""CAL""","""CASH"""
3,0,0.00,"11,000.00",0.00,"59,999.80",0.00,"6,140.00",12.00,12.00,"""P94_109_143""","""P131_33_167""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""
4,0,0.00,"16,000.00",0.00,"40,000.00",0.00,"2,556.60",24.00,24.00,"""P24_27_36""","""P194_82_174""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
5,0,0.00,"62,000.00","2,161.10","45,335.69",471.80,"3,745.33",18.44,18.44,"""P85_114_140""","""P54_133_26""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""T""","""CAL""","""CASH"""
6,0,0.00,"37,000.00",0.00,"32,000.00",0.00,"4,189.60",24.00,24.00,"""P94_109_143""","""P82_154_182""","""P97_36_170""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""CAL""","""CASH"""


In [20]:
train_files_df.filter(pl.col("filename").str.contains("applprev_2"))

index,path,filename
i64,str,str
0,"""Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""


In [21]:
train_past_depth_two = (
    pl.scan_parquet(all_train_files[0])
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains(["num_group"]).cast(pl.UInt8),
        cs.string().cast(pl.Categorical)
    )
)
train_past_depth_two.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L
u32,u8,u8,cat,cat,cat
2,1,1,null,"""EMPLOYMENT_PHONE""",null
2,0,1,null,"""EMPLOYMENT_PHONE""",null
2,0,0,null,"""PRIMARY_MOBILE""",null
2,1,0,null,"""PRIMARY_MOBILE""",null
3,0,1,null,"""PRIMARY_MOBILE""",null


In [22]:
train_past_depth_two_transformed = (
    train_past_depth_two
    .pipe(select_imputable_cols)
    .pipe(rename_cols)
    .with_columns(
        cs.contains("Card").fill_null(value=cs.contains("Card").mode().first())
    )
    .filter((pl.col("num_group1") == 0) & (pl.col("num_group2") == 0))
)
train_past_depth_two_transformed.fetch().head()

case_id,num_group1,num_group2,cacccardblochreas_147M Card blocking reason.
u32,u8,u8,cat
2,0,0,null
3,0,0,null
4,0,0,null
5,0,0,null
6,0,0,null


## Static Files

In [23]:
train_files_df.filter(cs.contains("file").str.contains("train_static_0_"))

index,path,filename
i64,str,str
21,"""Downloads/credit_risk/train/train_static_0_0.parquet""","""train_static_0_0.parquet"""
16,"""Downloads/credit_risk/train/train_static_0_1.parquet""","""train_static_0_1.parquet"""


In [24]:
train_static_base = (
    pl.scan_parquet("Downloads/credit_risk/train/train_static_0_*.parquet")
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_static_base.fetch().head()

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,…,pctinstlsallpaidlate1d_3546856L,pctinstlsallpaidlate4d_3546849L,pctinstlsallpaidlate6d_3546844L,pmtnum_254L,posfpd10lastmonth_333P,posfpd30lastmonth_3976960P,posfstqpd30lastmonth_3976962P,price_1097A,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,bankacctype_710L,cardtype_51L,credtype_322L,disbursementtype_67L,dtlastpmtallstes_4499206D,firstclxcampaign_1125D,inittransactioncode_186L,lastapprcommoditycat_1041M,lastapprcommoditytypec_5251766M,lastcancelreason_561M,lastrejectcommoditycat_161M,lastrejectcommodtypec_5251769M,lastrejectreason_759M,lastrejectreasonclient_4145040M,lastst_736L,paytype1st_925L,paytype_783L,payvacationpostpone_4187118D,previouscontdistrict_112M,twobodfilling_608L,typesuite_864L,validfrom_1069D
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
0,null,null,"1,917.60",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,…,null,null,null,24.00,0.00,0.00,null,null,0.00,0.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""",null,null,"""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""OTHER""","""OTHER""",null,"""a55475b1""","""BO""",null,null
1,null,null,"3,134.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,…,null,null,null,18.00,0.00,0.00,null,null,0.00,0.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""",null,null,"""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,"""OTHER""","""OTHER""",null,"""a55475b1""","""BO""",null,null
2,null,null,"4,937.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,…,null,null,null,36.00,0.00,0.00,null,null,0.00,0.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""",null,null,"""CASH""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""D""","""OTHER""","""OTHER""",null,"""a55475b1""","""BO""","""AL""",null
3,null,null,"4,643.60",0.00,0.00,1.00,0.00,2.00,0.00,1.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00,0.00,1.00,null,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,null,…,null,null,null,12.00,0.00,0.00,null,null,1.00,1.00,null,null,0.00,0.00,null,null,null,"""CAL""","""GBA""",null,null,"""CASH""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""a55475b1""","""P94_109_143""","""a55475b1""","""D""","""OTHER""","""OTHER""",null,"""a55475b1""","""BO""","""AL""",null
4,null,null,"3,390.20",0.00,0.00,1.00,0.00,0.00,0.00,1.00,null,null,null,null,null,null,null,null,null,0.00,0.00,0.00,0.00,0.00

In [25]:
train_static_base_transformed = (
    train_static_base
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .with_columns(
        cs.categorical().fill_null(value=cs.categorical().mode().first()),
        cs.float().fill_null(strategy="mean")
    )
)
train_static_base_transformed.fetch().head()

case_id,annuity_780A Monthly annuity amount.,applicationcnt_361L Number of applications associated with the same email address as the client.,applications30d_658L Number of applications made by the client in the last 30 days.,applicationscnt_1086L Number of applications associated with the same phone number.,applicationscnt_464L Number of applications made in the last 30 days by other clients with the same employer as the applicant.,applicationscnt_629L Number of applications with the same employer in the last 7 days.,applicationscnt_867L Number of applications associated with the same mobile phone.,clientscnt12m_3712952L Number of clients that have used the same mobile phone as the applicant in the past 12 months.,clientscnt3m_3712950L Number of clients who have the same mobile phone number in the last 3 months.,clientscnt6m_3712949L Total number of clients who have used the same mobile number in the last 6 months.,clientscnt_100L Number of applications with matching employer's phone and client's.,clientscnt_1022L Number of clients sharing the same mobile phone.,clientscnt_1071L Number of applications where the alternative phone number matches that of the client.,clientscnt_1130L Number of applications where client's phone number matches the alternative phone contact.,clientscnt_157L Number of clients whose employer has the same phone number as the client.,clientscnt_257L Number of clients that share an alternative phone number with the applicant.,clientscnt_304L Number of clients with the same phone number.,clientscnt_360L Number of clients that have the same alternative phone number and employer's phone number.,clientscnt_493L Number of clients with matching phone numbers for both the employer and the client.,clientscnt_533L Number of clients with same client's and alternative's phone number,clientscnt_887L Number of clients sharing the same employer's phone number.,clientscnt_946L Number of clients with matching mobile and employer's number.,credamount_770A Loan amount or credit card limit.,deferredmnthsnum_166L Number of deferred months.,disbursedcredamount_1113A Disbursed credit amount after consolidation.,downpmt_116A Amount of downpayment.,homephncnt_628L Number of distinct home phones on client's application.,mobilephncnt_593L Number of persons with the same mobile phone number.,numactivecreds_622L Number of active credits.,numactivecredschannel_414L Number of active credits.,numactiverelcontr_750L Number of active revolving credits.,numcontrs3months_479L Number of contracts in last 3 months.,numnotactivated_1143L Number of non-activated credits.,numpmtchanneldd_318L Number of previous loan contracts for the applicant that had direct debit as payment channel.,numrejects9m_859L Number of credit applications that were rejected in the last 9 months.,sellerplacecnt_915L Number of sellerplaces where the same client's document was used.,sellerplacescnt_216L Number of sellerplaces where the same client's mobile phone was used.,lastapprcommoditycat_1041M Commodity category of the last loan applications made by the applicant.,lastapprcommoditytypec_5251766M Commodity type of the last application.,lastcancelreason_561M Cancellation reason of the last application.,lastrejectcommoditycat_161M Category of commodity in the applicant's last rejected application.,lastrejectcommodtypec_5251769M Commodity type of the last rejected application.,lastrejectreason_759M Reason for rejection on the most recent rejected application.,lastrejectreasonclient_4145040M Reason for the client's last loan rejection.,previouscontdistrict_112M Contact district of the client's previous approved application.,credtype_322L Type of credit.,inittransactioncode_186L Transaction type of the initial credit transaction.,totaldebt_9A Total amount of debt.,annuitynextmonth_57A Next month's amount of annuity.,currdebt_22A Current debt amount of the client.,currdebtcredtyperange_828A Current amount of debt of the applicant.,numinstls_657L Number of instalments.,tot

In [26]:
train_files_df.filter(cs.contains("file").str.contains("static"))

index,path,filename
i64,str,str
21,"""Downloads/credit_risk/train/train_static_0_0.parquet""","""train_static_0_0.parquet"""
16,"""Downloads/credit_risk/train/train_static_0_1.parquet""","""train_static_0_1.parquet"""
3,"""Downloads/credit_risk/train/train_static_cb_0.parquet""","""train_static_cb_0.parquet"""


In [27]:
train_static_external =(
    pl.scan_parquet("Downloads/credit_risk/train/train_static_cb_0.parquet")
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_static_external.fetch().head()

case_id,contractssum_5085716L,days120_123L,days180_256L,days30_165L,days360_512L,days90_310L,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,riskassesment_940T,secondquarter_766L,thirdquarter_1082L,description_5085714M,education_1103M,education_88M,maritalst_385M,maritalst_893M,requesttype_4525192L,riskassesment_302T
u32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat
357,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,"6,301.40",null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
381,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,"4,019.60",null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
388,null,6.00,8.00,2.00,10.00,4.00,2.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,6.00,10.00,null,null,null,null,null,null,6.00,"14,548.00",null,3.00,5.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
405,null,0.00,0.00,0.00,1.00,0.00,0.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4.00,1.00,null,null,null,null,null,null,6.00,"10,498.24",null,2.00,0.00,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null
409,null,2.00,3.00,0.00,3.00,1.00,4.00,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.00,3.00,null,null,null,null,null,null,7.00,"6,344.88",null,0.00,4.00,"""a55475b1""","""717ddd49""","""a55475b1""","""a7fcb6e5""","""a55475b1""",null,null


In [28]:
train_static_external_transformed = (
    train_static_external
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .drop(cs.contains("birth"))
    .with_columns(
        cs.float().fill_null(strategy="mean")
    )
)
train_static_external_transformed.fetch().head()

case_id,description_5085714M Categorization of clients by credit bureau.,education_1103M Level of education of the client provided by external source.,education_88M Education level of the client.,maritalst_385M Marital status of the client.,maritalst_893M Marital status of the client,days120_123L Number of credit bureau queries for the last 120 days.,days180_256L Number of credit bureau queries for last 180 days.,days30_165L Number of credit bureau queries for the last 30 days.,days360_512L Number of Credit Bureau queries for last 360 days.,days90_310L Number of credit bureau queries for the last 90 days.,firstquarter_103L Number of results obtained from credit bureau in the first quarter.,fourthquarter_440L Number of results in fourth quarter.,numberofqueries_373L Number of queries to credit bureau.,secondquarter_766L Number of results in second quarter.,thirdquarter_1082L Number of results in third quarter.
u32,cat,cat,cat,cat,cat,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
357,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2.10,2.88,0.69,4.88,1.62,1.79,2.40,4.88,1.73,2.25
381,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",2.10,2.88,0.69,4.88,1.62,1.79,2.40,4.88,1.73,2.25
388,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",6.00,8.00,2.00,10.00,4.00,2.00,6.00,10.00,3.00,5.00
405,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",0.00,0.00,0.00,1.00,0.00,0.00,4.00,1.00,2.00,0.00
409,"""a55475b1""","""717ddd49""","""a55475b1""","""a7fcb6e5""","""a55475b1""",2.00,3.00,0.00,3.00,1.00,4.00,1.00,3.00,0.00,4.00


## Train Person

In [29]:
train_files_df.filter(pl.col("filename").str.contains("person"))

index,path,filename
i64,str,str
5,"""Downloads/credit_risk/train/train_person_1.parquet""","""train_person_1.parquet"""
30,"""Downloads/credit_risk/train/train_person_2.parquet""","""train_person_2.parquet"""


In [30]:
train_person_depth_one = (
    pl.scan_parquet(all_train_files[5])
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.by_name("num_group1").cast(pl.UInt8),
        cs.boolean(),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical()),
    )
)
train_person_depth_one.fetch().head()

case_id,num_group1,contaddr_matchlist_1032L,contaddr_smempladdr_334L,isreference_387L,remitter_829L,safeguarantyflag_411L,childnum_185L,mainoccupationinc_384A,personindex_1023L,persontype_1072L,persontype_792L,birth_259D,contaddr_district_15M,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,empl_employedtotal_800L,empl_industry_691L,empladdr_district_926M,empladdr_zipcode_114M,familystate_447L,gender_992L,housetype_905L,housingtype_772L,incometype_1044T,language1_981M,maritalst_703L,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,role_1084L,role_993L,sex_738L,type_25L
u32,u8,bool,bool,bool,bool,bool,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
0,0,false,false,null,null,true,null,"10,800.00",0.00,1.00,1.00,"""1986-07-01""","""P88_18_84""","""P167_100_165""","""P97_36_170""","""2017-09-15""","""MORE_FIVE""","""OTHER""","""P142_57_166""","""P167_100_165""","""MARRIED""",null,null,null,"""SALARIED_GOVT""","""P10_39_147""",null,"""P88_18_84""","""P167_100_165""",null,null,"""CL""",null,"""F""","""PRIMARY_MOBILE"""
0,1,null,null,null,false,null,null,null,1.00,1.00,4.00,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""SPOUSE""",null,"""EM""",null,null,"""PHONE"""
0,2,null,null,null,false,null,null,null,2.00,4.00,5.00,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""COLLEAGUE""","""SPOUSE""","""PE""",null,null,"""PHONE"""
0,3,null,null,null,null,null,null,null,null,5.00,null,null,"""a55475b1""","""a55475b1""","""a55475b1""",null,null,null,"""a55475b1""","""a55475b1""",null,null,null,null,null,"""a55475b1""",null,"""a55475b1""","""a55475b1""",null,"""COLLEAGUE""","""PE""",null,null,"""PHONE"""
1,0,false,false,null,null,true,null,"10,000.00",0.00,1.00,1.00,"""1957-08-01""","""P103_93_94""","""P176_37_166""","""P97_36_170""","""2008-10-29""","""MORE_FIVE""","""OTHER""","""P49_46_174""","""P160_59_140""","""DIVORCED""",null,null,null,"""SALARIED_GOVT""","""P10_39_147""",null,"""P103_93_94""","""P176_37_166""",null,null,"""CL""",null,"""M""","""PRIMARY_MOBILE"""


In [31]:
train_person_depth_one_transformed = (
    train_person_depth_one
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
    .with_columns(
        cs.categorical().fill_null(value=cs.categorical().mode().first()),
        cs.float().fill_null(strategy="mean")
    )
)
train_person_depth_one_transformed.fetch().head()

case_id,num_group1,contaddr_district_15M Zip code of a contact person's address.,contaddr_zipcode_807M Zip code of contact address.,education_927M Education level of the person.,empladdr_district_926M District where the employer's address is located.,empladdr_zipcode_114M Zipcode of employer's address.,language1_981M The primary language of the person.,registaddr_district_1083M District of person's registered address.,registaddr_zipcode_184M Registered address's zip code of a person.,persontype_1072L Person type.,role_1084L Type of contact role.,type_25L Contact type of a person.
u32,u8,cat,cat,cat,cat,cat,cat,cat,cat,f32,cat,cat
0,0,"""P88_18_84""","""P167_100_165""","""P97_36_170""","""P142_57_166""","""P167_100_165""","""P10_39_147""","""P88_18_84""","""P167_100_165""",1.00,"""CL""","""PRIMARY_MOBILE"""
0,1,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",1.00,"""EM""","""PHONE"""
0,2,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",4.00,"""PE""","""PHONE"""
0,3,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",5.00,"""PE""","""PHONE"""
1,0,"""P103_93_94""","""P176_37_166""","""P97_36_170""","""P49_46_174""","""P160_59_140""","""P10_39_147""","""P103_93_94""","""P176_37_166""",1.00,"""CL""","""PRIMARY_MOBILE"""


In [32]:
train_person_depth_two = (
    pl.scan_parquet(all_train_files[30])
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_person_depth_two.fetch().head()

case_id,num_group1,num_group2,addres_district_368M,addres_role_871L,addres_zip_823M,conts_role_79M,empls_economicalst_849M,empls_employedfrom_796D,empls_employer_name_740M,relatedpersons_role_762T
u32,u8,u8,cat,cat,cat,cat,cat,cat,cat,cat
5,0,0,"""a55475b1""",null,"""a55475b1""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
6,0,0,"""P55_110_32""","""CONTACT""","""P10_68_40""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""",null
6,0,1,"""P55_110_32""","""PERMANENT""","""P10_68_40""","""a55475b1""","""a55475b1""",null,"""a55475b1""",null
6,1,0,"""P204_92_178""","""CONTACT""","""P65_136_169""","""P38_92_157""","""P164_110_33""",null,"""a55475b1""","""OTHER_RELATIVE"""
6,1,1,"""P191_109_75""","""CONTACT""","""P10_68_40""","""P7_147_157""","""a55475b1""",null,"""a55475b1""","""OTHER_RELATIVE"""


In [33]:
train_person_depth_two_transformed = (
    train_person_depth_two
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
)
train_person_depth_two_transformed.fetch().head()

case_id,num_group1,num_group2,addres_district_368M District of the person's address.,addres_zip_823M Zip code of the address.,conts_role_79M Type of contact role of a person.,"empls_economicalst_849M The economical status of the person (num_group1 - person, num_group2 - employment).","empls_employer_name_740M Employer's name (num_group1 - person, num_group2 - employment)."
u32,u8,u8,cat,cat,cat,cat,cat
5,0,0,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
6,0,0,"""P55_110_32""","""P10_68_40""","""P38_92_157""","""P164_110_33""","""a55475b1"""
6,0,1,"""P55_110_32""","""P10_68_40""","""a55475b1""","""a55475b1""","""a55475b1"""
6,1,0,"""P204_92_178""","""P65_136_169""","""P38_92_157""","""P164_110_33""","""a55475b1"""
6,1,1,"""P191_109_75""","""P10_68_40""","""P7_147_157""","""a55475b1""","""a55475b1"""


In [34]:
train_files_df.filter(cs.by_name("filename").str.contains("other"))

index,path,filename
i64,str,str
24,"""Downloads/credit_risk/train/train_other_1.parquet""","""train_other_1.parquet"""


In [35]:
train_other = (
    pl.scan_parquet(all_train_files[24])
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt8),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_other.fetch().head()

case_id,num_group1,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A
u32,u8,f32,f32,f32,f32,f32
"43,801",0,"12,466.60","12,291.20",914.20,0.00,304.80
"43,991",0,"3,333.40","3,273.40",0.00,0.00,0.00
"44,001",0,"10,000.00","10,000.00",0.00,0.00,0.00
"44,053",0,0.00,0.00,"2,586.40",0.00,88.80
"44,130",0,63.80,60.80,0.00,0.00,0.00


In [36]:
train_other_transformed = (
    train_other
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
)
train_other_transformed.fetch().head()

case_id,num_group1,amtdebitincoming_4809443A Incoming debit card transactions amount.,amtdebitoutgoing_4809440A Outgoing debit card transactions amount.,amtdepositbalance_4809441A Deposit balance of client.,amtdepositincoming_4809444A Amount of incoming deposits to client's account.,amtdepositoutgoing_4809442A Amount of outgoing deposits from client's account.
u32,u8,f32,f32,f32,f32,f32
"43,801",0,"12,466.60","12,291.20",914.20,0.00,304.80
"43,991",0,"3,333.40","3,273.40",0.00,0.00,0.00
"44,001",0,"10,000.00","10,000.00",0.00,0.00,0.00
"44,053",0,0.00,0.00,"2,586.40",0.00,88.80
"44,130",0,63.80,60.80,0.00,0.00,0.00


In [37]:
train_files_df.filter(cs.by_name("filename").str.contains("registry"))

index,path,filename
i64,str,str
1,"""Downloads/credit_risk/train/train_tax_registry_a_1.parquet""","""train_tax_registry_a_1.parquet"""
22,"""Downloads/credit_risk/train/train_tax_registry_b_1.parquet""","""train_tax_registry_b_1.parquet"""
23,"""Downloads/credit_risk/train/train_tax_registry_c_1.parquet""","""train_tax_registry_c_1.parquet"""


In [38]:
train_first_registry = (
    pl.scan_parquet(all_train_files[1])
    .drop(cs.contains("date"))
    .pipe(rename_cols)
)
display(train_first_registry.fetch().head(2))
train_second_registry = (
    pl.scan_parquet(all_train_files[22])
    .drop(cs.contains("date"))
    .pipe(rename_cols)
)
display(train_second_registry.fetch().head(2))
train_third_registry = (
    pl.scan_parquet(all_train_files[23])
    .drop(cs.contains("date"))
    .pipe(rename_cols)
)
display(train_third_registry.fetch().head(2))
(
    pl.concat(
        [
            train_first_registry.select(cs.by_name('case_id').unique()),
            train_second_registry.select(cs.by_name("case_id").unique()),
            train_third_registry.select(cs.by_name("case_id").unique())
        ],
        how="vertical"
    )
    .with_context(
        train_base
    )
    .select(pl.col("case_id_base").is_in(pl.col("case_id")).mean().alias("total case_ids that are present in train base case ids"))
    .collect()
)

case_id,amount_4527230A Tax deductions amount tracked by the government registry.,name_4527232M Name of employer.,num_group1
i64,f64,str,i64
"28,631","1,946.00","""f980a1ea""",2
"28,631",711.00,"""f980a1ea""",3


case_id,amount_4917619A Tax deductions amount tracked by the government registry.,name_4917606M Name of employer.,num_group1
i64,f64,str,i64
"49,435","6,885.00","""6b730375""",7
"49,435","6,885.00","""6b730375""",1


case_id,employername_160M Employer's name.,num_group1,pmtamount_36A Tax deductions amount for credit bureau payments.
i64,str,i64,f64
357,"""c91b12ff""",5,"1,100.00"
357,"""c91b12ff""",1,"1,200.00"


total case_ids that are present in train base case ids
f64
0.66


In [39]:
train_files_df.filter(pl.col("filename").str.contains("_a_1_"))

index,path,filename
i64,str,str
11,"""Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""
14,"""Downloads/credit_risk/train/train_credit_bureau_a_1_1.parquet""","""train_credit_bureau_a_1_1.parquet"""
28,"""Downloads/credit_risk/train/train_credit_bureau_a_1_2.parquet""","""train_credit_bureau_a_1_2.parquet"""
9,"""Downloads/credit_risk/train/train_credit_bureau_a_1_3.parquet""","""train_credit_bureau_a_1_3.parquet"""


In [40]:
train_credit_bureau = (
    pl.scan_parquet("Downloads/credit_risk/train/train_credit_bureau_a_1_*.parquet")
    .drop(cs.contains("date"))
    .select(
        cs.by_name("case_id").cast(pl.UInt32),
        cs.contains("num_group").cast(pl.UInt16),
        cs.float().cast(pl.Float32),
        cs.string().cast(pl.Categorical)
    )
)
train_credit_bureau.fetch().head()

case_id,num_group1,annualeffectiverate_199L,annualeffectiverate_63L,contractsum_5085717L,credlmt_230A,credlmt_935A,debtoutstand_525A,debtoverdue_47A,dpdmax_139P,dpdmax_757P,instlamount_768A,instlamount_852A,interestrate_508L,monthlyinstlamount_332A,monthlyinstlamount_674A,nominalrate_281L,nominalrate_498L,numberofcontrsvalue_258L,numberofcontrsvalue_358L,numberofinstls_229L,numberofinstls_320L,numberofoutstandinstls_520L,numberofoutstandinstls_59L,numberofoverdueinstlmax_1039L,numberofoverdueinstlmax_1151L,numberofoverdueinstls_725L,numberofoverdueinstls_834L,outstandingamount_354A,outstandingamount_362A,overdueamount_31A,overdueamount_659A,overdueamountmax2_14A,overdueamountmax2_398A,overdueamountmax_155A,overdueamountmax_35A,periodicityofpmts_1102L,periodicityofpmts_837L,prolongationcount_1120L,prolongationcount_599L,residualamount_488A,residualamount_856A,totalamount_6A,totalamount_996A,totaldebtoverduevalue_178A,totaldebtoverduevalue_718A,totaloutstanddebtvalue_39A,totaloutstanddebtvalue_668A,classificationofcontr_13M,classificationofcontr_400M,contractst_545M,contractst_964M,description_351M,financialinstitution_382M,financialinstitution_591M,numberofoverdueinstlmaxdat_148D,numberofoverdueinstlmaxdat_641D,purposeofcred_426M,purposeofcred_874M,subjectrole_182M,subjectrole_93M
u32,u16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
388,0,null,null,null,null,null,"374,419.50",0.00,0.00,null,null,null,null,"7,811.45",null,null,null,2.00,1.00,null,60.00,null,54.00,0.00,null,0.00,null,null,"260,093.70",null,0.00,0.00,null,0.00,null,null,30.00,null,null,null,null,null,"268,897.62",0.00,0.00,"374,419.50",0.00,"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""",null,null,"""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
388,1,null,null,null,null,"135,806.00",null,null,0.00,null,"8,742.80",null,null,"8,742.80",null,null,null,null,null,null,null,null,null,0.00,null,0.00,null,null,null,null,0.00,0.00,null,0.00,null,null,null,null,null,null,"114,325.80",null,null,null,null,null,null,"""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""P204_66_73""",null,null,"""60c73645""","""a55475b1""","""a55475b1""","""a55475b1"""
388,2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,4,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""",null,null,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [41]:
train_credit_bureau_transformed = (
    train_credit_bureau
    .pipe(rename_cols)
    .pipe(select_imputable_cols)
)
train_credit_bureau_transformed.fetch().head()

case_id,num_group1,classificationofcontr_13M Classificiation of the active contract.,classificationofcontr_400M Classificiation of the closed contract.,contractst_545M Contract status.,contractst_964M Contract status of terminated credit contract.,description_351M Categorization of clients by credit bureau.,financialinstitution_382M Name of financial institution that is linked to a closed contract.,financialinstitution_591M Financial institution name of the active contract.,purposeofcred_426M Purpose of credit for active contract.,purposeofcred_874M Purpose of credit on a closed contract.,subjectrole_182M Subject role in active credit contract.,subjectrole_93M Subject role in closed credit contract.
u32,u16,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat
388,0,"""4408ff0f""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""55b002a9""","""96a8fdfe""","""a55475b1""","""ab3c25cf""","""ab3c25cf"""
388,1,"""ea6782cc""","""a55475b1""","""7241344e""","""a55475b1""","""a55475b1""","""a55475b1""","""P204_66_73""","""60c73645""","""a55475b1""","""a55475b1""","""a55475b1"""
388,2,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,3,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""
388,4,"""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1""","""a55475b1"""


In [47]:
(
    train_credit_bureau_transformed
    .select("case_id")
    .with_context(
        train_base.select("case_id_base")
    )
    .select(
        pl.col("case_id_base").is_in(pl.col('case_id')).mean()
    )
    .collect()
)

case_id_base
f64
0.91
